# COMED NITS

In [1]:
from icap.comed.comed import COMEDInterval
from icap.database.icapdatabase import ICapDatabase
from icap.results.results import Results

import pandas as pd
import numpy as np
from datetime import datetime
import tempfile
import os

In [2]:
fp = 'icap/database/icapdatabase.json'
conn = ICapDatabase(fp).connect()

In [3]:
cmd = COMEDInterval(conn)

In [4]:
cmd_icap = cmd.compute_icap()
#icap = cmd_icap.drop(labels=['DistLossFactor_y', 'TransLossFactor_y'], axis=1).copy()
#icap = icap.rename(columns={'DistLossFactor_x': 'DistLossFactor', 'TransLossFactor_x': 'TransLossFactor'})


In [ ]:
icap

In [ ]:
class RecordWriter:
    def __init__(self, records=None):
        assert(records is not None)
        self.records = records
        
        self.filename = 'comed_interval_nits.csv'
        self.path = os.path.join('/home/ubuntu/JustEnergy/', self.filename)
        
        
    def write(self):
        self.write_header()
        self.write_records()
        
    def write_header(self):
        
        header = 'PREMISEID, DSC, RUNDATE,'\
        'PJM CP DATE 1, PJM HOURENDING 1, PJM USAGE 1,'\
        'PJM CP DATE 2, PJM HOURENDING 2, PJM USAGE 2,'\
        'PJM CP DATE 3, PJM HOURENDING 3, PJM USAGE 3,'\
        'PJM CP DATE 4, PJM HOURENDING 4, PJM USAGE 4,'\
        'PJM CP DATE 5, PJM HOURENDING 5, PJM USAGE 5,'\
        'COMED CP DATE 1, COMED CP HOURENDING 1, COMED USAGE 1, COMED ZONAL 1,'\
        'COMED CP DATE 2, COMED CP HOURENDING 2, COMED USAGE 2, COMED ZONAL 2,'\
        'COMED CP DATE 3, COMED CP HOURENDING 3, COMED USAGE 3, COMED ZONAL 3,'\
        'COMED CP DATE 4, COMED CP HOURENDING 4, COMED USAGE 4, COMED ZONAL 4,'\
        'COMED CP DATE 5, COMED CP HOURENDING 5, COMED USAGE 5, COMED ZONAL 5,'\
        'DISTRIBUTION LOSS, TRANSMISSION LOSS, CUSTOMER DELTA, UFT, UFC, ACUSTCPL,'\
        'ACUSTPL, ICAP, NITS'
        
        with open(self.path, 'w') as fout:
                fout.write(header + os.linesep)
        return
    
    def write_records(self):
        with open(self.path, 'a+') as fout:
            for r in self.records:
                    fout.write(r.string_record + os.linesep)
        return

In [ ]:
class Record:
    def __init__(self, premise_id=None, year=None):
        # PJM CP Data
        self.premise_id = premise_id
        self.year = year
        self.run_date = datetime.now()
        self.pjm_cp_df = None
        
        
        # Comed CP Data
        self.comed_cp_df = None
        
        self.dsc = None  # Delivery Service Class
        self.string_record = None
        self.icap_df = None
        self.nits = None
    
    def compute_nits(self):
        assert(self.icap_df is not None)
        acustpl = self.icap_df.AcustPL.iloc[0]
        dist_loss = self.icap_df.DistLossFactor.iloc[0]
        trans_loss = self.icap_df.TransLossFactor.iloc[0]
        uft = self.icap_df.UFT.iloc[0]
        
        # nspl = ?
        self.nits =  acustpl * dist_loss * trans_loss * uft

    def append_empty_rows(self, df):
        # Get number of rows to add
        num_new_rows = 5 - df.shape[0]

        # Empty series to append dataframe
        empty = pd.Series([np.NaN for _ in range(df.shape[1])], index=df.columns, name='empty')
        for r in range(num_new_rows):
            df = df.append(empty)
        return df
                

    def format_df(self, df):
        if df.shape[0] > 5:
            return df.iloc[:5]
        elif df.shape[0] < 5:
            return self.append_empty_rows(df)
        else:
            return df
        
    def string_builder(self):
        assert(self.pjm_cp_df is not None)
        assert(self.comed_cp_df is not None)
        assert(self.icap_df is not None)
        if self.nits is None:
            self.compute_nits()
        
        rec = ''
        rec += '{premise_id}, {dsc}, {run_date},'.format(**self.__dict__)
        
        # PJM CP Data
        pjm = self.format_df(self.pjm_cp_df.sort_values(by='pjm_cp_date')) 
        for row in pjm.itertuples():
            _, cp, usage = row
            rec += '{cp}, {hour}, {usage},'.format(cp=cp, hour=None, usage=usage)
            
        # COMED CP Data
        comed = self.format_df(self.comed_cp_df.sort_values(by='comed_cp_date'))
        for row in comed.itertuples():
            _, cp, zonal, usage = row
            data = dict(cp=cp, hour=None, usage=usage, zonal=zonal)
            rec += '{cp}, {hour}, {usage}, {zonal},'.format(**data)
            
        # ICap Data
        rec += '{},'.format(self.icap_df.DistLossFactor.iloc[0])
        rec += '{},'.format(self.icap_df.TransLossFactor.iloc[0])
        rec += '{},'.format(self.icap_df.CustDelta.iloc[0])
        rec += '{},'.format(self.icap_df.UFT.iloc[0])
        rec += '{},'.format(self.icap_df.UFC.iloc[0])
        rec += '{},'.format(self.icap_df.AcustCPL.iloc[0])
        rec += '{},'.format(self.icap_df.AcustPL.iloc[0])
        rec += '{},'.format(self.icap_df.ICap.iloc[0])
        rec += '{}'.format(self.nits)
        
            
        self.string_record = rec

In [ ]:
# Initialize records using PJM CP data
records = list()
for k, g in cmd.pjm_records.groupby(['premise_id', 'year']):
    r = Record(*k) # premise, year 
    r.pjm_cp_df = g.drop(labels=['premise_id', 'year'], axis=1).copy() 
    records.append(r)
    
    


In [ ]:
# Populate records with COMED CP data
cmd_data = cmd.comed_records.copy()
for r in records:
    prem_data = cmd_data[(cmd_data.premise_id == r.premise_id) & (cmd_data.year == r.year)]
    data = prem_data.drop(labels=['premise_id', 'year'], axis=1)
    r.comed_cp_df = data.copy()

In [ ]:
# Add Delivery Service Class
for r in records:
    _dsc = cmd.dsc_records[cmd.dsc_records.premise_id == r.premise_id]['dsc'].values[0]
    r.dsc = _dsc

In [ ]:
# Add existing ICap Caluclations and factors
for r in records:
    prem = r.premise_id
    year = r.year
    
    df = icap[(icap.PremiseId == prem) & (cmd_icap.Year == year)]
    df = df.drop(labels=['PremiseId', 'Year', 'RateClass', 'Strata'], axis=1).copy()
    r.icap_df = df

In [ ]:
# Build string
for r in records:
    r.string_builder()
    

In [ ]:
rw = RecordWriter(records)
rw.write()

In [ ]:
type(records[0].icap_df.DistLossFactor.iloc[0].tolist())

In [ ]:
records[0].string_record